In [23]:
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import spacy
import re
import string
from tqdm import tqdm
from langdetect import detect, DetectorFactory, LangDetectException
from plotly.subplots import make_subplots
from styles import *

**Preprocesamiento de datos**


In [24]:
features_df = pd.read_csv("data/spotify_music_features.csv")
features_df.head()


,valence,year,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo
0,0.0594,1921,0.982,"['Sergei Rachmaninoff', 'James Levine', 'Berli...",0.279,831667,0.211,0,4BJqT0PrAfrxzMOxytFOIz,0.878000,10,0.665,-20.096,1,"Piano Concerto No. 3 in D Minor, Op. 30: III. ...",4,1921,0.0366,80.954
1,0.9630,1921,0.732,['Dennis Day'],0.819,180533,0.341,0,7xPhfUan2yNtyFG0cUWkt8,0.000000,7,0.160,-12.441,1,Clancy Lowered the Boom,5,1921,0.4150,60.936
2,0.0394,1921,0.961,['KHP Kridhamardawa Karaton Ngayogyakarta Hadi...,0.328,500062,0.166,0,1o6I8BglA6ylDMrIELygv1,0.913000,3,0.101,-14.850,1,Gati Bali,5,1921,0.0339,110.339
3,0.1650,1921,0.967,['Frank Parker'],0.275,210000,0.309,0,3ftBPsC5vPBKxYSee08FDH,0.000028,5,0.381,-9.316,1,Danny Boy,3,1921,0.0354,100.109
4,0.2530,1921,0.957,['Phil Regan'],0.418,166693,0.193,0,4d6HGyGT8e121BsdKmw9v6,0.000002,3,0.229,-10.096,1,When Irish Eyes Are Smiling,2,1921,0.0380,101.665


In [25]:
lyrics_df = pd.read_csv("data/lyrics_dataset.csv")
lyrics_df.head()

,id,artist,song_name,lyrics,decade
0,4Hhv2vrOTy89HFRcjU3QOx,Etta James,At Last,[Verse 1] At last My love has come along My lo...,1960
1,0zGLlXbHlrAyBN1x6sY0rb,Etta James,A Sunday Kind Of Love,[Verse 1] I want a Sunday kind of love A love ...,1960
2,4ukUoXLuFzMixyZyabSGc4,Ella Fitzgerald,Sleigh Ride,"Just hear those sleigh bells jingling, ring, t...",1960
3,65irrLqfCMRiO3p87P4C0D,Ella Fitzgerald,Frosty The Snowman,Who's debonair with the tall silk hat Muffler ...,1960
4,2x6pbpjVGjiWCcH89IK8AX,Neil Sedaka,Breaking Up Is Hard to Do,"[Intro] To do do, down dooby doo down down Com...",1960


In [ ]:
# Mergear el datset de feautures con el dataset de letras
df = pd.merge(features_df, lyrics_df, on='id', how='inner')
df.drop(columns=['artist'], inplace=True)

In [ ]:
# Analisis general
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21000 entries, 0 to 20999
Data columns (total 23 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   valence           21000 non-null  float64
 1   year              21000 non-null  int64  
 2   acousticness      21000 non-null  float64
 3   artists           21000 non-null  object 
 4   danceability      21000 non-null  float64
 5   duration_ms       21000 non-null  int64  
 6   energy            21000 non-null  float64
 7   explicit          21000 non-null  int64  
 8   id                21000 non-null  object 
 9   instrumentalness  21000 non-null  float64
 10  key               21000 non-null  int64  
 11  liveness          21000 non-null  float64
 12  loudness          21000 non-null  float64
 13  mode              21000 non-null  int64  
 14  name              21000 non-null  object 
 15  popularity        21000 non-null  int64  
 16  release_date      21000 non-null  object

In [ ]:
df.describe()

,valence,year,acousticness,danceability,duration_ms,energy,explicit,instrumentalness,key,liveness,loudness,mode,popularity,speechiness,tempo,decade
count,21000.000000,21000.000000,21000.000000,21000.000000,2.100000e+04,21000.000000,21000.000000,21000.000000,21000.000000,21000.000000,21000.000000,21000.000000,21000.000000,21000.000000,21000.000000,21000.00000
mean,0.550191,1992.579286,0.313420,0.571222,2.330169e+05,0.597384,0.123381,0.051304,5.248000,0.195333,-9.127724,0.711429,44.599000,0.077762,120.902419,1988.11619
std,0.249421,17.142356,0.307474,0.165824,9.865979e+04,0.233498,0.328882,0.165373,3.538865,0.165206,4.172120,0.453109,14.980242,0.089648,29.697367,17.14722
min,0.000000,1960.000000,0.000001,0.000000,3.058300e+04,0.000099,0.000000,0.000000,0.000000,0.009670,-36.090000,0.000000,0.000000,0.000000,0.000000,1960.00000
25%,0.349000,1978.000000,0.033200,0.458000,1.832930e+05,0.426000,0.000000,0.000000,2.000000,0.094075,-11.750250,0.000000,34.000000,0.033200,97.541750,1970.00000
50%,0.558000,1994.000000,0.203000,0.577000,2.217055e+05,0.613000,0.000000,0.000021,5.000000,0.130000,-8.453000,1.000000,44.000000,0.043700,118.928500,1990.00000
75%,0.756000,2007.250000,0.568000,0.690000,2.646930e+05,0.789000,0.000000,0.002762,8.000000,0.251000,-5.918000,1.000000,55.000000,0.074400,139.986250,2000.00000
max,1.000000,2020.000000,0.996000,0.978000,5.403500e+06,0.999000,1.000000,0.999000,11.000000,0.997000,1.483000,1.000000,99.000000,0.960000,216.538000,2020.00000


In [ ]:
df.isnull().sum()

valence             0
year                0
acousticness        0
artists             0
danceability        0
duration_ms         0
energy              0
explicit            0
id                  0
instrumentalness    0
key                 0
liveness            0
loudness            0
mode                0
name                0
popularity          0
release_date        0
speechiness         0
tempo               0
artist              0
song_name           0
lyrics              1
decade              0
dtype: int64

In [ ]:
cols = ['id'] + [col for col in df.columns if col != 'id']
df = df[cols]

In [ ]:
# Reemplazar la columna duration_ms por duration_min
df['duration_min'] = (df['duration_ms'] / 60000).round(2)
df.drop(columns=['duration_ms'], inplace=True)


In [ ]:
# Crear columna decade
df['decade'] = (df['year'] // 10) * 10
df.drop(columns=['year', 'release_date'], inplace=True)

In [ ]:
# Filtrar las canciones de la década de 1960 en adelante
df = df[df['decade'] >= 1960]

In [ ]:
# Crear variable binaria de éxito duradero con percentil 80
threshold = df['popularity'].quantile(0.80)
print(f"Umbral de éxito duradero (percentil 80): {threshold:.2f}")

df['was_trending'] = (df['popularity'] >= threshold).astype(int)

Umbral de éxito duradero (percentil 80): 58.00


**Preprocesamiento de letras de canciones**

In [ ]:
! python -m spacy download en_core_web_sm
! python -m spacy download es_core_news_sm


     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     --------------------------------------  12.6/12.8 MB 81.8 MB/s eta 0:00:01
     --------------------------------------- 12.8/12.8 MB 59.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
     --------------------------------------  12.8/12.9 MB 64.0 MB/s eta 0:00:01
     --------------------------------------- 12.9/12.9 MB 55.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
DetectorFactory.seed = 0
nlp_en = spacy.load("en_core_web_sm", disable=["ner", "parser"])
nlp_es = spacy.load("es_core_news_sm", disable=["ner", "parser"])

In [ ]:
def detect_language(text):
    try:
        return detect(text) if isinstance(text, str) and len(text.strip()) > 5 else "unknown"
    except LangDetectException:
        return "unknown"

print("Detectando idiomas ...")
df["language"] = df["lyrics"].apply(detect_language)

Detectando idiomas ...


In [ ]:
df["language"].value_counts()

language
en         18247
es          1984
pt           147
fr           108
ja            76
ko            57
it            44
de            38
tl            33
hi            26
ru            24
id            24
so            23
sw            22
ro            16
tr            16
nl            13
ar            13
fa            12
ca            10
pl             7
cy             7
he             7
zh-tw          6
sv             5
et             4
af             4
unknown        4
vi             3
no             3
el             3
zh-cn          2
sk             2
fi             2
lv             1
hr             1
bn             1
uk             1
da             1
pa             1
hu             1
sl             1
Name: count, dtype: int64

In [ ]:
# Filtra solo las canciones en inglés y español
df = df[df['language'].isin(['en', 'es'])]

In [ ]:
# Limpieza general de texto
def clean_text(text):
    if not isinstance(text, str):
        return ""
    text = text.lower()                                     # Minúsculas
    text = re.sub(r'\[.*?\]', '', text)                     # Eliminar [Verse], [Chorus], etc.
    text = re.sub(r'http\S+|www\S+', '', text)              # Eliminar URLs
    text = re.sub(r'\d+', '', text)                         # Eliminar números
    text = text.translate(str.maketrans('', '', string.punctuation))  # Eliminar puntuación
    text = re.sub(r'\s+', ' ', text).strip()                # Quitar espacios extra
    return text

df["lyrics"] = df["lyrics"].apply(clean_text)

In [ ]:
# Lematizacion y elimancion de stopwords
def preprocess_text(df):
    results = []

    for lang, nlp in [('en', nlp_en), ('es', nlp_es)]:
        subset_idx = df[df['language'] == lang].index
        subset = df.loc[subset_idx, 'lyrics']

        if subset.empty:
            continue

        texts = [re.sub(r'[^a-záéíóúñü\s]', '', str(t).lower()) for t in subset]
        stopwords = nlp.Defaults.stop_words

        docs = nlp.pipe(texts, batch_size=32, n_process=4)
        tokens_per_doc = []

        for doc in tqdm(docs, total=len(texts), desc=f"Lematizando {lang}"):
            tokens = [
                token.lemma_ for token in doc
                if token.is_alpha and token.lemma_.lower() not in stopwords and len(token) > 2
            ]
            tokens_per_doc.append(tokens)

        results.extend(zip(subset_idx, tokens_per_doc))

    tokens_dict = dict(results)
    df['tokens'] = df.index.map(tokens_dict)
    return df

df = preprocess_text(df)



Lematizando es: 100%|██████████| 1984/1984 [00:30<00:00, 64.56it/s] 


In [ ]:
# Detección de valores atípicos
num_cols = [
    'valence', 'acousticness', 'danceability', 
    'duration_min', 'energy', 'instrumentalness'
]

colors = px.colors.qualitative.Plotly

rows, cols = 1, 6
fig = make_subplots(rows=rows, cols=cols)

for i, col in enumerate(num_cols):
    r = (i // cols) + 1
    c = (i % cols) + 1
    fig.add_trace(
        go.Box(
            y=df[col],
            marker_color=colors[i % len(colors)],
            showlegend=True,
            name=col
        ),
        row=r, col=c
    )

fig.update_layout(title='Valores atípicos en características numéricas')
fig = boxplot_style(fig, n_cols=cols, width=1100)
fig.show()

In [ ]:
# Detección de valores atípicos
num_cols = [
    'key', 'liveness', 'loudness',
    'popularity', 'speechiness', 'tempo'
]

colors = px.colors.qualitative.Plotly

rows, cols = 1, 6
fig = make_subplots(rows=rows, cols=cols)

for i, col in enumerate(num_cols):
    r = (i // cols) + 1
    c = (i % cols) + 1
    fig.add_trace(
        go.Box(
            y=df[col],
            marker_color=colors[i % len(colors)],
            showlegend=True,
            name=col
        ),
        row=r, col=c
    )

fig.update_layout(title='Valores atípicos en características numéricas')
fig = boxplot_style(fig, n_cols=cols, width=1100)
fig.show()

In [ ]:
df.to_csv('data/dataset_processed.csv', index=False)